In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Here we import necessary libraries of scikit-learn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

**Reading data**

In [ ]:
trn = pd.read_json(open("../input/train.json", "r"))
tst = pd.read_json(open("../input/test.json", "r"))

In [ ]:
#let's see how big these datasets are
print("Train set: ", trn.shape)
print("Test set: ", tst.shape)


In [ ]:
list(trn.columns.values)

In [ ]:
trn.head()

In [ ]:
trn["num_photos"] = trn["photos"].apply(len)
trn["num_features"] = trn["features"].apply(len)
trn["num_description_words"] = trn["description"].apply(lambda x: len(x.split(" ")))
trn["created"] = pd.to_datetime(trn["created"])
trn["created_year"] = trn["created"].dt.year
trn["created_month"] = trn["created"].dt.month
trn["created_day"] = trn["created"].dt.day

In [ ]:
list(trn.columns.values)

In [ ]:
'''fts = ['bathrooms','bedrooms','building_id','created','description','display_address','features',
         'interest_level','latitude','listing_id','longitude','manager_id','photos','price',
         'street_address','num_photos','num_features','num_description_words',
         'created_year','created_month','created_day']'''
fts = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]
X = trn[fts]
y = trn["interest_level"]

**Model Training**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

**Working with test data**

In [ ]:
tst["num_photos"] = tst["photos"].apply(len)
tst["num_features"] = tst["features"].apply(len)
tst["num_description_words"] = tst["description"].apply(lambda x: len(x.split(" ")))
tst["created"] = pd.to_datetime(tst["created"])
tst["created_year"] = tst["created"].dt.year
tst["created_month"] = tst["created"].dt.month
tst["created_day"] = tst["created"].dt.day
X = tst[fts]

y = clf.predict_proba(X)

In [ ]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx

In [ ]:
sub = pd.DataFrame()
sub["listing_id"] = tst["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
print sub
#sub.to_csv("submission_rf.csv", index=False)